In [ ]:
import pandas as pd
import numpy as np
import altair as alt

# STEP 1: Read the data  
Output: the dictionary __dataset__

In [ ]:
facebook_df = pd.read_csv("/content/drive/MyDrive/Data Vis 1 64/Task4 (Implement)/aggr_fb_df_v2.csv")
youtube_df = pd.read_csv("/content/drive/MyDrive/Data Vis 1 64/Task4 (Implement)/yt_v4_after_midterm.csv")
twitter_df = pd.read_csv("/content/drive/MyDrive/Data Vis 1 64/Task4 (Implement)/twitter_v2.csv")
instagram_df = pd.read_csv("/content/drive/MyDrive/Data Vis 1 64/Task4 (Implement)/aggr_ig_df_v2.csv")

dataset = {"Facebook": facebook_df, "Youtube": youtube_df, "Twitter": twitter_df, "Instagram": instagram_df}
color_picker = {"Facebook": "#3258a8", "Youtube": "#f23c0a", "Twitter": "#11ddf7", "Instagram": "#983bf5"}

In [ ]:
facebook_df.head()

,account_id,total_num_post,q1,q2,q3,q4,monday,tuesday,wednesday,thursday,friday,saturday,sunday,morning,afternoon,evening,night,change_fan_per_day,percent_change_fan,posts_per_day,posts_per_hour,post_freq,likes_per_post,loves_per_post,wows_per_post,hahas_per_post,angrys_per_post,sads_per_post,comments_per_post,shares_per_post,reaction_ratio_per_post,engagement_ratio_per_post,purchase_intent_ratio_per_post
0,100106458256720,2369,0.928662,31.743352,28.281976,39.046011,13.803293,14.014352,13.423385,13.085690,13.381173,14.689743,17.602364,26.677923,49.641199,18.488814,5.192064,0.000000,0.000000,8.872659,0.369184,3.196955,120.306458,4.619671,0.180245,0.086957,0.023217,0.036302,26.662305,89.946813,0.001826,0.003526,0.001796
1,1001088136643378,491,49.694501,6.924644,34.419552,8.961303,53.564155,14.867617,7.331976,3.462322,7.128310,3.054990,10.590631,13.034623,23.421589,62.932790,0.610998,2.953782,10.297349,2.063025,0.085939,16.332362,8.287169,1.083503,0.120163,0.014257,0.026477,0.474542,0.156823,1.509165,0.001428,0.001665,0.003055
2,1002461093150254,713,17.391304,40.532959,16.830295,25.245442,12.342216,15.147265,8.835905,19.915849,11.781206,17.251052,14.726508,19.214586,31.697055,46.704067,2.384292,1.105740,0.193126,3.909366,0.162665,12.231839,544.669243,109.493045,0.457496,16.238794,0.838485,10.731066,15.904946,35.445904,0.006535,0.007027,0.006921
3,100315104676476,1294,36.939722,16.692427,18.624420,27.743431,8.423493,11.591963,14.142195,14.837713,18.315301,17.233385,15.455951,3.477589,44.281298,52.241113,0.000000,1.409341,1.149657,2.074176,0.086418,6.756812,295.372185,3.847682,3.633113,15.147020,0.739073,4.377483,14.729801,144.652980,0.004286,0.006398,0.006484
4,100353938099291,755,37.615894,24.635762,15.629139,22.119205,9.536424,8.741722,9.271523,11.258278,10.728477,43.841060,6.622517,66.490066,28.344371,4.900662,0.264901,1.118243,7.572638,4.854730,0.201682,11.433283,17.306193,0.363953,0.080724,0.221990,0.001392,0.022269,2.291580,2.416841,0.007824,0.009870,0.014073


In [ ]:
youtube_df.head()

,Unnamed: 0,account_id,total_num_post,q1,q2,q3,q4,monday,tuesday,wednesday,thursday,friday,saturday,sunday,morning,afternoon,evening,night,change_fan_per_day,percent_change_fan,posts_per_day,post_freq,reaction_ratio_per_post,engagement_ratio_per_post,likes_per_post,dislikes_per_post,views_per_post,comments_per_post,shares_per_post
0,0,UC--rAfJFoqz2VFCcrNkgJVw,34,35.294118,17.647059,26.470588,20.588235,29.411765,14.705882,11.764706,29.411765,2.941176,2.941176,8.823529,20.588235,26.470588,47.058824,5.882353,0.000000,0.000000,0.099415,248.800227,0.012279,0.013017,92.117647,1.323529,3736.970588,5.617647,241.294118
1,1,UC-2PvCjARBOTjuHgEmtm-6A,364,24.725275,24.725275,25.274725,25.274725,14.285714,14.285714,14.560440,14.285714,14.285714,14.010989,14.285714,99.450549,0.274725,0.274725,0.000000,0.000000,0.000000,0.997260,24.132246,0.007635,0.008339,166.837912,1.901099,4368.530220,15.552198,43.420330
2,2,UC-3mhofjFcKzrP9ZKNq4CIw,3,33.333333,0.000000,66.666667,0.000000,33.333333,33.333333,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,66.666667,33.333333,0.000000,0.000000,0.015228,2375.963611,0.015228,0.018613,6.000000,0.000000,67.333333,1.333333,1.666667
3,3,UC-DQaiKtl6j3Sa3X22ZDT5g,126,20.634921,23.809524,23.809524,31.746032,9.523810,12.698413,6.349206,11.904762,11.111111,10.317460,38.095238,0.000000,7.142857,92.857143,0.000000,223.463687,24.316109,0.351955,68.743331,0.008368,0.008684,3076.238095,25.111111,111867.825397,116.992063,45.611111
4,4,UC-Jqlu1ShAYkdWi9eZYCJpg,70,38.571429,35.714286,14.285714,11.428571,12.857143,10.000000,7.142857,35.714286,28.571429,4.285714,1.428571,20.000000,77.142857,2.857143,0.000000,0.000000,0.000000,0.200573,121.637890,0.008284,0.008828,11.042857,0.471429,772.042857,0.757143,39.628571


In [ ]:
twitter_df.head()

,Unnamed: 0,account_id,total_num_post,q1,q2,q3,q4,monday,tuesday,wednesday,thursday,friday,saturday,sunday,morning,afternoon,evening,night,change_fan_per_day,percent_change_fan,posts_per_day,post_freq,favorite_ratio_per_post,engagement_ratio_per_post,reply_per_post,retweet_per_post
0,0,1002112842974162945,31,32.258065,35.483871,12.903226,19.354839,6.451613,6.451613,16.129032,3.225806,9.677419,32.258065,25.806452,29.032258,25.806452,45.161290,0.000000,0.000000,0.000000,0.105802,235.149565,0.083517,0.169089,0.225806,63.096774
1,1,100651983,20,10.000000,0.000000,20.000000,70.000000,10.000000,15.000000,5.000000,15.000000,10.000000,25.000000,20.000000,20.000000,40.000000,40.000000,0.000000,1.743590,2.775636,0.056980,443.915278,0.003754,0.028425,1.100000,547.500000
2,2,1006994652711432193,200,28.500000,23.500000,23.500000,24.500000,15.000000,16.500000,15.500000,13.000000,12.000000,15.000000,13.000000,8.000000,25.000000,65.000000,2.000000,15.318560,62.698413,0.554017,43.540702,0.075156,0.255855,2.740000,1944.585000
3,3,1014180338690486272,624,42.628205,23.076923,17.467949,16.826923,16.987179,17.467949,16.506410,16.185897,16.185897,9.935897,6.730769,22.435897,53.846154,23.557692,0.160256,28.632597,111.715887,1.723757,13.963623,0.003877,0.009603,0.690705,71.158654
4,4,1015259058775851009,20,30.000000,55.000000,15.000000,0.000000,15.000000,15.000000,20.000000,15.000000,10.000000,5.000000,20.000000,10.000000,35.000000,50.000000,5.000000,0.116379,1.989683,0.086207,293.658070,0.020022,0.070778,0.150000,69.400000


In [ ]:
instagram_df.head()

,account_id,total_num_post,q1,q2,q3,q4,monday,tuesday,wednesday,thursday,friday,saturday,sunday,morning,afternoon,evening,night,change_fan_per_day,percent_change_fan,posts_per_day,posts_per_hour,post_freq,likes_per_post,comments_per_post,views_per_post,shares_per_post,engagement_ratio_per_post,message_length_per_post
0,1019027411,1,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,586.000000,1.000000,0.000000,0.0,0.009113,3.000000
1,10247924,383,20.365535,24.804178,22.193211,32.637076,13.054830,13.838120,13.838120,17.232376,15.143603,14.360313,12.532637,18.798956,46.475196,34.725849,0.000000,0.000000,0.000000,1.049315,0.043709,22.938650,121.261097,2.198433,47.101828,0.0,0.002257,25.741514
2,10268293051,39,56.410256,23.076923,5.128205,15.384615,25.641026,20.512821,12.820513,7.692308,15.384615,7.692308,10.256410,23.076923,53.846154,15.384615,7.692308,0.000000,0.000000,0.110482,0.004596,223.291053,90.871795,2.897436,11.358974,0.0,0.016709,8.769231
3,10330514,379,24.802111,21.899736,30.079156,23.218997,10.290237,15.303430,14.511873,15.831135,11.345646,15.831135,16.886544,32.981530,62.532982,4.485488,0.000000,77.616438,78.072037,1.038356,0.043259,23.177970,2182.424802,4.308707,494.620053,0.0,0.053065,25.005277
4,1039601,237,24.050633,31.223629,22.362869,22.362869,11.814346,15.611814,13.924051,16.455696,10.126582,15.189873,16.877637,13.080169,54.852321,29.957806,2.109705,26.323288,3.637644,0.649315,0.027016,37.171402,3837.679325,34.843882,1411.354430,0.0,0.014563,21.679325


## STEP 2: (Post) Activities across Platforms  
__Question of Interest__: If one user was picked at random, how many posts per  day could be expected in each platform? Did this number change across 2020?  
__Variable of Interest__: average number of posts per day by each user  
__Aggregation__: The mean of variable of interest over all users

In [ ]:
total_post_dictionary = {}
num_rows = 0
for platform in list(dataset.keys()):
  mean_total_post = dataset[platform]["total_num_post"].mean() / 365
  total_post_dictionary[num_rows] = [platform, mean_total_post, color_picker[platform]]
  num_rows += 1

total_posts = pd.DataFrame.from_dict(total_post_dictionary, orient = "index", columns = ["platform", "value", "color"])

In [ ]:
post_activities_dictionary = {}
num_rows = 0

for platform in list(dataset.keys()):
  for quarter in range(1, 5):
    mean_post_per_user = (dataset[platform]["q"+str(quarter)] * dataset[platform]["total_num_post"] / (100*90)).mean()
    pct_change = 0
    if quarter != 1:
      prev_val = post_activities_dictionary[num_rows - 1][2]
      pct_change = (mean_post_per_user - prev_val) * 100 / prev_val
    post_activities_dictionary[num_rows] = [platform, "Q" + str(quarter) + ":2020", mean_post_per_user, pct_change, color_picker[platform]]
    num_rows += 1


post_activities = pd.DataFrame.from_dict(post_activities_dictionary, orient = "index", columns = ["platform", "quarter", "value", "percentage_change", "color"])

In [ ]:
num_posts_bar = alt.Chart(total_posts).mark_bar(size = 50).encode(
    x = alt.X("platform", axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "posts : day")),
    color= alt.Color("platform", scale = alt.Scale(domain = list(color_picker.keys()), range = list(color_picker.values())))
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User"
)

In [ ]:
post_activities_line = alt.Chart(post_activities).mark_line().encode(
                  x=alt.X('quarter',axis = alt.Axis(title = '', labelAngle = 0)),
                  y= alt.Y('value', axis=alt.Axis(title='posts: day')),
                  color= alt.Color("platform", scale = alt.Scale(domain = list(color_picker.keys()), range = list(color_picker.values())))
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User over Time"
)

In [ ]:
alt.hconcat(num_posts_bar, post_activities_line, title = alt.TitleParams("Social Media Analytics", align = "left", fontSize = 20, dx = 400))

alt.HConcatChart(...)

### Interaction Example : Selection

In [ ]:
selector = alt.selection_single(empty='all', fields=['platform'])
color_scale =  alt.Color("platform:N", scale = alt.Scale(domain = list(color_picker.keys()), range = list(color_picker.values())), legend = None)

num_posts_bar = alt.Chart(total_posts).mark_bar(size = 50).encode(
    x = alt.X("platform", axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "posts : day")),
    color=alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User"
).add_selection(
    selector
)

In [ ]:
post_activities_line = alt.Chart(post_activities).mark_line().encode(
                  x=alt.X('quarter',axis = alt.Axis(title = '', labelAngle = 0)),
                  y= alt.Y('value', axis=alt.Axis(title='posts: day'), scale = alt.Scale(domain = (0, 5.5))),
                  color= alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User over Time"
)


text = alt.Chart(post_activities).mark_text(dx=0, dy=-5).encode(
    x=alt.X("quarter"),
    y=alt.Y('value'),
    text=alt.condition(selector, alt.Text('value:Q', format='.2f'), alt.value(""))
)

In [ ]:
alt.hconcat(num_posts_bar, post_activities_line + text)

alt.HConcatChart(...)

## Step 3: Engagement per Post by Categories  
__Question of Interest__: If one user was picked at random, how many reactions/comments/shares could be expected on a post?  
__Variable of Interest__: average number of reactions/comments/shares per post by each user  
__Aggregation__: The mean of variable of interest over all users  


In [ ]:
col_dict = {"Facebook": [["likes_per_post",	"loves_per_post",	"wows_per_post",	"hahas_per_post",	"angrys_per_post",	"sads_per_post"],	["comments_per_post"],	["shares_per_post"]],
                "Youtube": [["likes_per_post",	"dislikes_per_post",	"views_per_post"],	["comments_per_post"],	["shares_per_post"]],
                "Twitter": [["favorite_ratio_per_post"],	["reply_per_post"],	["retweet_per_post"]],
                "Instagram": [["likes_per_post",	"views_per_post"],	["comments_per_post"],	["shares_per_post"]]}
total_engagement_dictionary = {}
num_rows = 0
aggs = {0: "reactions_per_post", 1: "comments_per_post", 2: "shares_per_post"}
for platform in list(dataset.keys()):
  for i in range(3):
    mean  = np.mean(dataset[platform][col_dict[platform][i]], axis = 0)
    order = i
    total_engagement_dictionary[num_rows] = [platform, aggs[i], sum(mean), color_picker[platform], order]
    num_rows += 1

total_engagements = pd.DataFrame.from_dict(total_engagement_dictionary, orient = "index", columns = ["platform", "attribute", "value", "color", "order"])

In [ ]:
engagement_line = alt.Chart(total_engagements).mark_line(point = True, size = 5).encode(
    x = alt.X("attribute", sort = ["reactions_per_post", "comments_per_post", "shares_per_post"], axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "")),
    color = "platform",
).properties(
    width = 400,
    height = 300,
)

In [ ]:
input_dropdown = alt.binding_select(options=list(dataset.keys()))
selector = alt.selection_single(fields=['platform'], bind=input_dropdown, name='Select')
color_scale =  alt.Color("platform:N", scale = alt.Scale(domain = list(color_picker.keys()), range = list(color_picker.values())), legend = None)

num_posts_bar = alt.Chart(total_posts).mark_bar(size = 50).encode(
    x = alt.X("platform", axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "posts : day")),
    color=alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User"
)

post_activities_line = alt.Chart(post_activities).mark_line().encode(
                  x=alt.X('quarter',axis = alt.Axis(title = '', labelAngle = 0)),
                  y= alt.Y('value', axis=alt.Axis(title='posts: day'), scale = alt.Scale(domain = (0, 5.5))),
                  color= alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User over Time"
)

text = alt.Chart(post_activities).mark_text(dx=0, dy=-5).encode(
    x=alt.X("quarter"),
    y=alt.Y('value'),
    text=alt.condition(selector, alt.Text('value:Q', format='.2f'), alt.value(""))
)

panel1 = alt.hconcat(num_posts_bar, post_activities_line + text)

engagement_line = alt.Chart(total_engagements).mark_line(point = True, size = 5).encode(
    x = alt.X("attribute", sort = ["reactions_per_post", "comments_per_post", "shares_per_post"], axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "")),
    color = alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
).add_selection(
    selector
)

text2 = alt.Chart(total_engagements).mark_text(dx=0, dy=-5).encode(
    x=alt.X("attribute", sort = ["reactions_per_post", "comments_per_post", "shares_per_post"], ),
    y=alt.Y('value'),
    text=alt.condition(selector, alt.Text('value:Q', format='.2f'), alt.value(""))
)

alt.vconcat(panel1, engagement_line + text2)

alt.VConcatChart(...)

## Step 4: Social Media Metrics by User
4.1 Percentage Change in Fans  
__Question of Interest__: How many users have experienced an increase/decrease in the number of fans?  
__Variable of Interest__: Percentage change in fans from the first post in 2020 to the last post in 2020 for each user  
__Aggregation__: binning users into decrease in fans, small to moderate increase in fans (less than mean percentage change) and large increase in fans (greater than or equal to mean percentage change)  
4.2 Engagement Ratio  
__Question of Interest__: How many users have experienced a high engagement ratio?  
__Variable of Interest__: average of ratio between number of engagements and number of fans in each post by user  
__Aggregation__: binning users into low to medium engagement ratio (less than mean engagement ratio) and high engagement ratio (greater than or equal to mean engagement ratio)


In [ ]:
engagement_dictionary = {}
num_rows = 0
for platform in list(dataset.keys()):
  engagemean = dataset[platform]["engagement_ratio_per_post"].mean()
  greatermean = sum(dataset[platform]["engagement_ratio_per_post"] >= engagemean)
  engagement_dictionary[num_rows] = [platform, "greater than mean", greatermean, color_picker[platform]]
  num_rows += 1
 
  engagemean = dataset[platform]["engagement_ratio_per_post"].mean()
  lessermean = sum(dataset[platform]["engagement_ratio_per_post"] < engagemean)
  engagement_dictionary[num_rows] = [platform,  "less than mean", lessermean, color_picker[platform]]
  num_rows += 1
    
engagement = pd.DataFrame.from_dict(engagement_dictionary, orient = "index", columns = ["platform", "category", "value", "color"])

In [ ]:
percentage_dictionary = {}
num_rows = 0
for platform in list(dataset.keys()):
  percentengagemean = dataset[platform]["percent_change_fan"].mean()
  percentgreatermean = sum(dataset[platform]["percent_change_fan"] >= percentengagemean)
  percentage_dictionary[num_rows] = [platform, "percent great than mean",percentgreatermean, color_picker[platform]]
  num_rows += 1
    
  percentengagemean = dataset[platform]["percent_change_fan"].mean()
  percentlessermean = sum(dataset[platform]["percent_change_fan"] < percentengagemean)
  percentage_dictionary[num_rows] = [platform,"percent less than mean", percentlessermean, color_picker[platform]]
  num_rows += 1  
    
  percentlesserzero = sum(dataset[platform]["percent_change_fan"] < 0)
  percentage_dictionary[num_rows] = [platform, "percent less zero",percentlesserzero, color_picker[platform]]
  num_rows += 1
 
percentage = pd.DataFrame.from_dict(percentage_dictionary, orient = "index", columns = ["platform", "category", "value", "color"])

In [ ]:
engagement

,platform,category,value,color
0,Facebook,greater than mean,414,#3258a8
1,Facebook,less than mean,1670,#3258a8
2,Youtube,greater than mean,16,#f23c0a
3,Youtube,less than mean,1015,#f23c0a
4,Twitter,greater than mean,13,#11ddf7
5,Twitter,less than mean,490,#11ddf7
6,Instagram,greater than mean,173,#983bf5
7,Instagram,less than mean,837,#983bf5


In [ ]:
percentage

,platform,category,value,color
0,Facebook,percent great than mean,446,#3258a8
1,Facebook,percent less than mean,1638,#3258a8
2,Facebook,percent less zero,325,#3258a8
3,Youtube,percent great than mean,1,#f23c0a
4,Youtube,percent less than mean,1018,#f23c0a
5,Youtube,percent less zero,20,#f23c0a
6,Twitter,percent great than mean,10,#11ddf7
7,Twitter,percent less than mean,493,#11ddf7
8,Twitter,percent less zero,62,#11ddf7
9,Instagram,percent great than mean,198,#983bf5


In [ ]:
input_dropdown = alt.binding_select(options=list(dataset.keys()))
selector = alt.selection_single(fields=['platform'], bind=input_dropdown, name='Select')
color_scale2 =  alt.Color("category:N", scale = alt.Scale(domain = ["less than mean", "greater than mean", "percent less zero", "percent less than mean", "percent great than mean"], 
                                                          range = ["#8bfa1b", "#097a2b", "#6205ad", "#8bfa1b", "#097a2b"]))

engagement_bar = alt.Chart(engagement).mark_bar(size = 20).encode(
    x = alt.X("sum(value)", axis = alt.Axis(title = "Number of User", labelAngle = 0)),
    y = alt.Y("platform", axis = alt.Axis(title = "")),
    color= alt.condition(selector, color_scale2, alt.value("lightgray"), legend = None),
    order = alt.Order("category", sort = "descending"),
    tooltip=[
        alt.Tooltip('value:Q', title="Number of users"),
        alt.Tooltip('category:N', title = "category")
    ]
).properties(
    width = 400,
    height = 150,
    title = "Engagement Ratio Per Post"
).add_selection(selector)



In [ ]:
percentage_bar = alt.Chart(percentage).mark_bar(size = 20).encode(
    x = alt.X("sum(value)", axis = alt.Axis(title = "Number of User", labelAngle = 0)),
    y = alt.Y("platform", axis = alt.Axis(title = "")),
    color= alt.condition(selector, color_scale2, alt.value("lightgray"), legend = None),

).properties(
    width = 400,
    height = 150,
    title = "Percent Change of Fan"
)

In [ ]:
alt.vconcat(engagement_bar, percentage_bar)

alt.VConcatChart(...)

## 5. Final Product

In [ ]:


input_dropdown = alt.binding_select(options=list(dataset.keys()))
selector = alt.selection_single(fields=['platform'], bind=input_dropdown, name='Select')
color_scale =  alt.Color("platform:N", scale = alt.Scale(domain = list(color_picker.keys()), range = list(color_picker.values())), legend = None)

num_posts_bar = alt.Chart(total_posts).mark_bar(size = 50).encode(
    x = alt.X("platform", axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "posts : day")),
    color=alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User"
)

post_activities_line = alt.Chart(post_activities).mark_line().encode(
                  x=alt.X('quarter',axis = alt.Axis(title = '', labelAngle = 0)),
                  y= alt.Y('value', axis=alt.Axis(title='posts: day'), scale = alt.Scale(domain = (0, 5.5))),
                  color= alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
    title = "Average Number of Posts Daily per User over Time"
)

text = alt.Chart(post_activities).mark_text(dx=0, dy=-5).encode(
    x=alt.X("quarter"),
    y=alt.Y('value'),
    text=alt.condition(selector, alt.Text('value:Q', format='.2f'), alt.value(""))
)

panel1 = alt.hconcat(num_posts_bar, post_activities_line + text)

engagement_scatter = alt.Chart(total_engagements).mark_line(point = True, size = 5).encode(
    x = alt.X("attribute", sort = ["reactions_per_post", "comments_per_post", "shares_per_post"], axis = alt.Axis(title = "", labelAngle = 0)),
    y = alt.Y("value", axis = alt.Axis(title = "")),
    color = alt.condition(selector, color_scale, alt.value("lightgray"), legend = None),
).properties(
    width = 400,
    height = 300,
).add_selection(
    selector
)

text2 = alt.Chart(total_engagements).mark_text(dx=0, dy=-5).encode(
    x=alt.X("attribute", sort = ["reactions_per_post", "comments_per_post", "shares_per_post"], ),
    y=alt.Y('value'),
    text=alt.condition(selector, alt.Text('value:Q', format='.2f'), alt.value(""))
)

alt.vconcat(panel1, engagement_scatter + text2)